In [1]:
import numpy as np
import numpy.linalg
import scipy.io
import meshplot as mp
from igl import *
import meshio
import h5py

import sys
sys.path.append('./ShellUtils')
%load_ext autoreload
%autoreload 2
import Gen, TriMesh, Visual, Geometry

In [17]:
InputH5 = '/Users/ziyizhang/Projects/Zebrafish/data/cem2-NewGlobalDisp.h5'
idxArray = np.array([1, 2, 3, 6, 12, 28], dtype=int)  # index of markers to align
                                                  # cannot be too small or too symmetric

In [18]:
# read re-analysis H5 file
with h5py.File(InputH5, 'r+') as f:
    print("Keys: %s" % f.keys())

    E = f['E'].value[()]
    F = f['F'].value[()]
    Nverts = f['Nverts'].value[()]
    V = f['V'].value[()]
    discr_order = f['discr_order'].value[()]
    frames = f['frames'].value[()]
    imgCols = f['imgCols'].value[()]
    imgRows = f['imgRows'].value[()]
    is_linear = f['is_linear'].value[()]
    layerPerImg = f['layerPerImg'].value[()]
    markerRCMap_mat = f['markerRCMap_mat'].value[()]
    max_tet_vol = f['max_tet_vol'].value[()]
    n_refs = f['n_refs'].value[()]
    nu = f['nu'].value[()]
    offset = f['offset'].value[()]
    radius_edge_ratio = f['radius_edge_ratio'].value[()]
    resolutionX = f['resolutionX'].value[()]
    resolutionY = f['resolutionY'].value[()]
    resolutionZ = f['resolutionZ'].value[()]
    upsample = f['upsample'].value[()]
    vismesh_rel_area = f['vismesh_rel_area'].value[()]

    VV = np.zeros((frames, Nverts, 3))
    for i in range(frames):
        VV[i, :, :] = V[i*Nverts:(i+1)*Nverts, :]

Keys: <KeysViewHDF5 ['E', 'F', 'Nverts', 'V', 'discr_order', 'frames', 'imgCols', 'imgRows', 'is_linear', 'layerPerImg', 'markerRCMap_mat', 'max_tet_vol', 'n_refs', 'nu', 'offset', 'radius_edge_ratio', 'resolutionX', 'resolutionY', 'resolutionZ', 'upsample', 'vismesh_rel_area']>


In [18]:
# re-store global displacement
#dispTxt = '/Users/ziyizhang/Projects/tmp/meanDisp.txt'
#meanDisp = np.loadtxt(dispTxt)
#VV_raw = np.array([VV[i, :, :]+meanDisp[i, :].reshape(1, 3) for i in range(VV.shape[0])])
VV_raw = VV

In [19]:
# The area to estimate global displacement
# idxArray = np.array([1, 2, 3, 6, 12], dtype=int)
alignPosArray = np.array([VV_raw[i, idxArray, :] for i in range(VV_raw.shape[0])])

In [20]:
# Visualize the selected index array (before alignment)
# red is frame-0, blue is frame-"frame"
frame = VV_raw.shape[0]-1
plt = mp.plot(VV_raw[0], Geometry.f2e(F))
plt.add_edges(VV_raw[frame], Geometry.f2e(F))
plt.add_points(alignPosArray[0], shading={"point_color": "red", "point_size": 26})
plt.add_points(alignPosArray[frame], shading={"point_color": "blue", "point_size": 20})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(159.10311…

3

In [22]:
# do alignment
VV_new = np.zeros((VV_raw.shape[0], VV_raw.shape[1], VV_raw.shape[2]))
alignPosArray_post = np.zeros((alignPosArray.shape[0], alignPosArray.shape[1], alignPosArray.shape[2]))
for f in range(VV_raw.shape[0]):
    #print(f)
    s, R, T = igl.procrustes(alignPosArray[f], alignPosArray[0], False, False)
    #print(R)
    #print(T)
    assert(s==1.0)
    #tt = np.matmul(VV_raw[0], R) + T.reshape(1, 3)
    #print("    ->   ", np.min(tt - VV_raw[f]))
    VV_new[f, :, :] = np.matmul(VV_raw[f], R) + T.reshape(1, 3)
    alignPosArray_post[f, :, :] = np.matmul(alignPosArray[f], R) + T.reshape(1, 3)

In [23]:
# Visualize the selected index array (after rotation+translation alignment)
# red is frame-0, blue is frame-"frame"
frame = VV_raw.shape[0]-1
plt = mp.plot(VV_raw[0], Geometry.f2e(F))
plt.add_edges(VV_new[frame], Geometry.f2e(F))
plt.add_points(alignPosArray[0], shading={"point_color": "red", "point_size": 26})
plt.add_points(alignPosArray_post[frame], shading={"point_color": "blue", "point_size": 20})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(159.10311…

3

In [24]:
# write result to new H5 file
# read re-analysis H5 file
OutputH5 = InputH5[:-3] + "-NewGlobalDisp.h5"

# get V_new
V_new = VV_new.reshape(Nverts*frames, 3)
VV_ = np.zeros((frames, Nverts, 3))
for i in range(frames):
    VV_[i, :, :] = V_new[i*Nverts:(i+1)*Nverts, :]
assert(np.min(VV_-VV_new) == 0)

with h5py.File(OutputH5, 'w') as f:

    f.create_dataset('E', data=E)
    f.create_dataset('F', data=F)
    f.create_dataset('Nverts', data=Nverts)
    f.create_dataset('V', data=V_new)
    f.create_dataset('discr_order', data=discr_order)
    f.create_dataset('frames', data=frames)
    f.create_dataset('imgCols', data=imgCols)
    f.create_dataset('imgRows', data=imgRows)
    f.create_dataset('is_linear', data=is_linear)
    f.create_dataset('layerPerImg', data=layerPerImg)
    f.create_dataset('markerRCMap_mat', data=markerRCMap_mat)
    f.create_dataset('max_tet_vol', data=max_tet_vol)
    f.create_dataset('n_refs', data=n_refs)
    f.create_dataset('nu', data=nu)
    f.create_dataset('offset', data=offset)
    f.create_dataset('radius_edge_ratio', data=radius_edge_ratio)
    f.create_dataset('resolutionX', data=resolutionX)
    f.create_dataset('resolutionY', data=resolutionY)
    f.create_dataset('resolutionZ', data=resolutionZ)
    f.create_dataset('upsample', data=upsample)
    f.create_dataset('vismesh_rel_area', data=vismesh_rel_area)

    VV = np.zeros((frames, Nverts, 3))
    for i in range(frames):
        VV[i, :, :] = V[i*Nverts:(i+1)*Nverts, :]

In [19]:
## Add point
adjList = np.array([86, 80, 93, 107, 110, 96])
adjList2 = np.array([77, 78, 87, 99, 104, 92])
VV_addPt = np.zeros((frames, Nverts+2, 3))
for i in range(frames):
    newPt = np.mean(VV[i, adjList, :], axis=0)
    newPt2 = np.mean(VV[i, adjList2, :], axis=0)
    VV_addPt[i, :, :] = np.vstack((VV[i, :, :], np.vstack((newPt, newPt2))))

newTri = np.array([
    [adjList[0], Nverts, adjList[1]], 
    [adjList[1], Nverts, adjList[2]], 
    [adjList[2], Nverts, adjList[3]], 
    [adjList[3], Nverts, adjList[4]], 
    [adjList[4], Nverts, adjList[5]], 
    [adjList[5], Nverts, adjList[0]],
    [adjList2[0], Nverts+1, adjList2[1]], 
    [adjList2[1], Nverts+1, adjList2[2]], 
    [adjList2[2], Nverts+1, adjList2[3]], 
    [adjList2[3], Nverts+1, adjList2[4]], 
    [adjList2[4], Nverts+1, adjList2[5]], 
    [adjList2[5], Nverts+1, adjList2[0]]
])
F_addPt = np.vstack((F, newTri))

In [20]:
## Add point
# write result to new H5 file
# read re-analysis H5 file
OutputH5 = InputH5[:-3] + "-AddVertex.h5"

# get V_new
V_new = VV_addPt.reshape((Nverts+2)*frames, 3)

with h5py.File(OutputH5, 'w') as f:

    f.create_dataset('E', data=E)
    f.create_dataset('F', data=F_addPt)
    f.create_dataset('Nverts', data=Nverts+2)
    f.create_dataset('V', data=V_new)
    f.create_dataset('discr_order', data=discr_order)
    f.create_dataset('frames', data=frames)
    f.create_dataset('imgCols', data=imgCols)
    f.create_dataset('imgRows', data=imgRows)
    f.create_dataset('is_linear', data=is_linear)
    f.create_dataset('layerPerImg', data=layerPerImg)
    f.create_dataset('markerRCMap_mat', data=markerRCMap_mat)
    f.create_dataset('max_tet_vol', data=max_tet_vol)
    f.create_dataset('n_refs', data=n_refs)
    f.create_dataset('nu', data=nu)
    f.create_dataset('offset', data=offset)
    f.create_dataset('radius_edge_ratio', data=radius_edge_ratio)
    f.create_dataset('resolutionX', data=resolutionX)
    f.create_dataset('resolutionY', data=resolutionY)
    f.create_dataset('resolutionZ', data=resolutionZ)
    f.create_dataset('upsample', data=upsample)
    f.create_dataset('vismesh_rel_area', data=vismesh_rel_area)